In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

mnist = datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = Sequential([
  Flatten(input_shape=(28,28)),
  Dense(32, activation='relu'),
  Dense(16, activation='relu'),
  Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

11490434/11490434 [==============================] - 0s 0us/step


## 3.5.5 클래스 가중치 및 샘플 가중치

In [2]:
import pandas as pd

pd.DataFrame(y_train).value_counts()

1    6742
7    6265
3    6131
2    5958
9    5949
0    5923
6    5918
8    5851
4    5842
5    5421
dtype: int64

In [3]:
weight = {
    0: 1.0,
    1: 1.0,
    2: 1.0,
    3: 1.0,
    4: 1.0,
    5: 1.2,
    6: 1.0,
    7: 1.0,
    8: 1.0,
    9: 1.0,
}

In [4]:
model.fit(x_train, y_train, 
          class_weight=weight, 
          batch_size=32, 
          epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3882 - accuracy: 0.8886
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1821 - accuracy: 0.9480
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1458 - accuracy: 0.9575
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1256 - accuracy: 0.9633
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1093 - accuracy: 0.9676


In [5]:
import numpy as np

weight = np.ones(shape=(len(y_train),))
weight[y_train == 5] = 1.2

print('sample_weight shape:', weight.shape)

model.fit(x_train, y_train, 
          sample_weight = weight, 
          batch_size=32, 
          epochs=5)

sample_weight shape: (60000,)
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0989 - accuracy: 0.9703
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0889 - accuracy: 0.9742
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0812 - accuracy: 0.9761
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0764 - accuracy: 0.9769
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0706 - accuracy: 0.9785


## 3.5.6 콜백 사용하기

In [6]:
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

mnist = datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = Sequential([
  Flatten(input_shape=(28,28)),
  Dense(32, activation='relu'),
  Dense(16, activation='relu'),
  Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

callback_EarlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [8]:
history = model.fit(x_train, y_train,
                    validation_split=0.2,
                    epochs=20, batch_size=1,
                    callbacks=callback_EarlyStopping,
                    verbose=1)

Epoch 1/20
48000/48000 [==============================] - 146s 3ms/step - loss: 0.2879 - accuracy: 0.9130 - val_loss: 0.2066 - val_accuracy: 0.9423
Epoch 2/20
48000/48000 [==============================] - 155s 3ms/step - loss: 0.1717 - accuracy: 0.9500 - val_loss: 0.1682 - val_accuracy: 0.9553
Epoch 3/20
48000/48000 [==============================] - 150s 3ms/step - loss: 0.1470 - accuracy: 0.9583 - val_loss: 0.1504 - val_accuracy: 0.9563
Epoch 4/20
48000/48000 [==============================] - 157s 3ms/step - loss: 0.1306 - accuracy: 0.9632 - val_loss: 0.1589 - val_accuracy: 0.9577
Epoch 5/20
48000/48000 [==============================] - 156s 3ms/step - loss: 0.1236 - accuracy: 0.9650 - val_loss: 0.1642 - val_accuracy: 0.9588
Epoch 6/20
48000/48000 [==============================] - 153s 3ms/step - loss: 0.1148 - accuracy: 0.9674 - val_loss: 0.1751 - val_accuracy: 0.9576
Epoch 7/20
48000/48000 [==============================] - 147s 3ms/step - loss: 0.1041 - accuracy: 0.9706 - val_

In [9]:
print( history.history.keys(), '\n' )

history.history['val_loss']

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy']) 



[0.20664815604686737,
 0.16818439960479736,
 0.15039092302322388,
 0.15885354578495026,
 0.1641886979341507,
 0.17514801025390625,
 0.19101880490779877,
 0.19881244003772736]

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

callback_ModelCheckpoint = ModelCheckpoint(filepath='./ModelCheckpoint',
                            monitor='val_loss',
                            save_weights_only=True,
                            save_best_only=True,
                            verbose=1)

history = model.fit(x_train, y_train,
                    validation_split=0.2,
                    epochs=20, batch_size=32,
                    callbacks=[callback_ModelCheckpoint],
                    verbose=1)

Epoch 1/20
1500/1500 [==============================] - ETA: 0s - loss: 0.0621 - accuracy: 0.9822
Epoch 1: val_loss improved from inf to 0.14687, saving model to ./ModelCheckpoint
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0621 - accuracy: 0.9822 - val_loss: 0.1469 - val_accuracy: 0.9672
Epoch 2/20
1493/1500 [============================>.] - ETA: 0s - loss: 0.0472 - accuracy: 0.9859
Epoch 2: val_loss improved from 0.14687 to 0.14088, saving model to ./ModelCheckpoint
1500/1500 [==============================] - 7s 4ms/step - loss: 0.0472 - accuracy: 0.9859 - val_loss: 0.1409 - val_accuracy: 0.9682
Epoch 3/20
1498/1500 [============================>.] - ETA: 0s - loss: 0.0427 - accuracy: 0.9872
Epoch 3: val_loss did not improve from 0.14088
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0427 - accuracy: 0.9872 - val_loss: 0.1462 - val_accuracy: 0.9685
Epoch 4/20
1494/1500 [============================>.] - ETA: 0s - loss: 0.0393 - accuracy: 0

In [11]:
loss, accuracy = model.evaluate(x_test, y_test)
print('\nTest accuracy:', accuracy)
print('Test loss:', loss)

model.load_weights(filepath='./ModelCheckpoint')
loss, accuracy = model.evaluate(x_test, y_test)
print('\nTest accuracy:', accuracy)
print('Test loss:', loss)

313/313 [==============================] - 1s 3ms/step - loss: 0.2067 - accuracy: 0.9653

Test accuracy: 0.9653000235557556
Test loss: 0.20665156841278076
313/313 [==============================] - 1s 2ms/step - loss: 0.1241 - accuracy: 0.9699

Test accuracy: 0.9699000120162964
Test loss: 0.12412606179714203


In [12]:
from tensorflow.keras.callbacks import TensorBoard

callback_TensorBoard = TensorBoard(log_dir='./logs', profile_batch=5)

history = model.fit(x_train, y_train,
                    validation_split=0.2,
                    epochs=20, batch_size=32,
                    callbacks=[callback_TensorBoard],
                    verbose=0)